In [1]:
!pip install SPARQLWrapper


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# Set up the SPARQL wrapper
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)


In [2]:


def save_gpx(qid:str):
  query = f"""
  #Wielrenners
  #title: Wielrenners
  SELECT DISTINCT ?race ?raceQid ?raceLabel ?startPointLabel ?startPointLongitude ?startPointLatitude  ?endPointLabel ?endPointLongitude ?endPointLatitude WHERE {{
    ?race wdt:P1346|wdt:P2321|wdt:P710 wd:{qid};

          
          
            wdt:P1427 ?startPoint;
            wdt:P1444 ?endPoint.
    ?startPoint wdt:P17 ?startPointCountry ;
                wdt:P625 ?startPointCoords .
    BIND(geof:latitude(?startPointCoords) AS ?startPointLatitude)
      BIND(geof:longitude(?startPointCoords) AS ?startPointLongitude)
    ?endPoint wdt:P17 ?endPointCountry ;
              wdt:P625 ?endPointCoords .
      BIND(geof:latitude(?endPointCoords) AS ?endPointLatitude)
      BIND(geof:longitude(?endPointCoords) AS ?endPointLongitude)
      BIND(STRAFTER(STR(?race), STR(wd:)) AS ?raceQid) 
    
  
    FILTER(?endPoint != ?startPoint && (?endPointCountry IN ( wd:Q31, wd:Q55 ) || ?endPointCountry IN ( wd:Q31, wd:Q55 )) )
    FILTER(?startPoint != wd:Q207095)
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "nl,en,fr,mul". }}
  }}
  """


  # Specify the query and format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and get the results
  results = sparql.query().convert()

  # Parse and print the results
  for result in results["results"]["bindings"]:
      race = result["raceQid"]["value"]
      body = {"coordinates":[[float(result["startPointLongitude"]["value"]),float(result["startPointLatitude"]["value"])],[float(result["endPointLongitude"]["value"]),float(result["endPointLatitude"]["value"])]]}
      
      print(race)
      headers = {
          'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
          'Authorization': '5b3ce3597851110001cf62489354ef714b2f4f2ca26b24494639a68c',
          'Content-Type': 'application/json; charset=utf-8'
      }
      call = requests.post('https://api.openrouteservice.org/v2/directions/cycling-regular/gpx', json=body, headers=headers)

      print(call.status_code, call.reason)
      with open(f'data/{race}.xml', 'w') as file:
          print(call.text, file=file)
# Q234158
# Q536470
# Q454963
# Q925964
for qid in ["Q925964"]:
   save_gpx(qid)

KeyboardInterrupt: 

In [3]:
import io
import json
import os
import re
import subprocess

import rdflib
import requests
from rdflib import Graph

sparql_anything_jar = "./sparql-anything-0.8.1.jar"

In [11]:
def create_query(qid):
    return f"""
    PREFIX fx: <http://sparql.xyz/facade-x/ns/> 
    PREFIX xyz: <http://sparql.xyz/facade-x/data/> 
    select ?lon ?lat
    WHERE {{
        SERVICE <x-sparql-anything:location=/home/lennert/hackalod/data/{qid}.xml> {{
        ?s xyz:lon ?lon .
        ?s xyz:lat ?lat

    }}
    }}
    """

In [14]:
import os

# Path to the folder containing the XML files
folder_path = 'data/'

# Get all XML files in the folder
xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

# Extract filenames without extensions
filenames_without_extension = [os.path.splitext(f)[0] for f in xml_files]

# Print the result
for filename in filenames_without_extension:
    print(filename)

    try:
        output = subprocess.run(
            [
                "java",
                "-jar",
                sparql_anything_jar,
                "-q",
                create_query(filename),
                "-v",
                f"-o lat_long_{filename}.csv"
            ],
            capture_output=True,
            check=True,
            universal_newlines=True,
        )
    except subprocess.CalledProcessError as e:
        print(e.stderr)
        print(e.stdout)
        raise e

# transf_data_graph.parse(data=output.stdout, format="turtle")   
print(output.stdout) 
print(output.stderr)

Q2093189
Q30899285
Q2488680
Q2486046
Q27506806
Q1116673
Q2995901
Q20107672
Q57778491
Q2410375
Q3478508
Q936387
Q20731452
Q3069353
Q4543091
Q3486951
Q4703206
Q2007239
Q27481596
Q2224084
Q107572399
Q23701739
Q2143208
Q2526723
Q30509759
Q19832815

[main] INFO com.github.sparqlanything.cli.SPARQLAnything - SPARQL anything



In [ ]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 25.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import os

# Path to the folder
folder_path = 'data'

# List of strings to check in filenames
# Q234158
  # wd:Q2995901
  # wd:Q23701739
  
# Q536470
  # wd:Q3486951
  # wd:Q2488680
  # wd:Q2526723
  # wd:Q27506806
# Q454963
  # wd:Q20107672
  # wd:Q19832815
  # wd:Q2143208
  # wd:Q4703206
# Q925964
  # Q27481596
  #  wd:Q23701739
required_strings = ['Q2995901', 'Q23701739', 'Q3486951', 'Q2488680', 'Q2526723', 'Q27506806', 'Q20107672', 'Q19832815', 'Q2143208', 'Q4703206', 'Q23701739']


# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Full path of the file
    file_path = os.path.join(folder_path, filename)

    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        # Check if any of the required strings are in the filename
        if not any(req_str in filename for req_str in required_strings):
            # If none of the strings are found, delete the file
            os.remove(file_path)
            print(f"Deleted: {filename}")
        else:
            print(f"Kept: {filename}")


Deleted: Q2093189.xml
Deleted:  lat_long_Q30899285.csv
Kept:  lat_long_Q2526723.csv
Deleted: Q30899285.xml
Kept: Q2488680.xml
Deleted: Q2486046.xml
Kept:  lat_long_Q3486951.csv
Kept: Q27506806.xml
Deleted: Q1116673.xml
Deleted:  lat_long_Q3478508.csv
Kept: Q2995901.xml
Deleted:  lat_long_Q2093189.csv
Kept: Q20107672.xml
Kept:  lat_long_Q19832815.csv
Deleted:  lat_long_Q2224084.csv
Deleted:  lat_long_Q3069353.csv
Deleted: Q57778491.xml
Deleted:  lat_long_Q20731452.csv
Deleted:  lat_long_Q1116673.csv
Deleted: Q2410375.xml
Deleted: Q3478508.xml
Deleted:  lat_long_Q30509759.csv
Deleted: Q936387.xml
Kept:  lat_long_Q27506806.csv
Deleted: Q20731452.xml
Deleted:  lat_long_Q936387.csv
Deleted: Q3069353.xml
Deleted: Q4543091.xml
Kept: Q3486951.xml
Kept:  lat_long_Q2143208.csv
Deleted: netherlands.tsv
Kept:  lat_long_Q2488680.csv
Kept: Q4703206.xml
Deleted: Q2007239.xml
Deleted: Q27481596.xml
Deleted: belgium.tsv
Deleted: Q2224084.xml
Deleted: Q107572399.xml
Kept: Q23701739.xml
Deleted:  lat_lon

In [27]:
import os
import pandas as pd

# Specify the folder containing CSV files
folder_path = 'data/'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Dictionary to store longitude and latitude data from each file
lon_lat_data = {}

# Loop through each CSV file
for file in csv_files:
    # Construct the full file path
    file_path = os.path.join(folder_path, file)
    
    # Read the CSV file using pandas
    df = pd.read_csv(file_path)
    
    # Check if the columns 'lon' and 'lat' exist
    if 'lon' in df.columns and 'lat' in df.columns:
        # Extract longitude and latitude
        lon_lat_data[file] = df[['lon', 'lat']].values.tolist()
    else:
        print(f"Columns 'lon' and 'lat' not found in {file}")

# Print the results
g = Graph()
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import SDO, RDF
import requests
import json
for file, data in lon_lat_data.items():
    for dat in data:
        uri = URIRef(f"http://hackalod.org/{dat[1]},{dat[0]}")
        lat = Literal(dat[1])
        lon = Literal(dat[0])
        g.add((uri, SDO.longitude, lon))
        g.add((uri, SDO.latitude, lat))
        url = f"http://127.0.0.1:80/r/{dat[0]}/{dat[1]}.js"

        payload = ""
        response = requests.request("GET", url, data=payload)

        result = json.loads(response.text)
        qid = result['results'][0]['wikidata']
        name = result['results'][0]['name']
        lit_name = Literal(name)
        if not qid:
            continue
        qidiri = URIRef(f'http://www.wikidata.org/entity/{qid}')
        g.add((qidiri, SDO.geo, uri))
        g.add((qidiri, SDO.name, lit_name))
        g.add((qidiri, RDF.type, SDO.GeoCoordinates))


g.serialize(destination="graph.ttl")

<Graph identifier=N2488a1753fd644fa93ce29719108a9b5 (<class 'rdflib.graph.Graph'>)>

In [28]:


def get_info(qid:str):
  query = f"""
  #Wielrenners
  #title: Wielrenners
  SELECT DISTINCT  ?person ?personLabel ?image  WHERE {{
    BIND(wd:{qid} as ?person)
    # ?race wdt:P1346|wdt:P2321|wdt:P710 ?person;
    OPTIONAL {{
      ?person wdt:P18 ?image
    }}
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "nl,en,fr,mul". }}
  }}
  """


  # Specify the query and format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and get the results
  results = sparql.query().convert()

  # Parse and print the results
  return results["results"]["bindings"]
  for result in results["results"]["bindings"]:
      race = result["raceQid"]["value"]
# Q234158
  # wd:Q2995901
  # wd:Q23701739
  
# Q536470
  # wd:Q3486951
  # wd:Q2488680
  # wd:Q2526723
  # wd:Q27506806
# Q454963
  # wd:Q20107672
  # wd:Q19832815
  # wd:Q2143208
  # wd:Q4703206
# Q925964
  # Q27481596
  #  wd:Q23701739	
for qid in ["Q925964", "Q234158", "Q536470", "Q454963"]:
  results = get_info(qid)
  print(results)
  result= results[0]
  uri = URIRef(result['person']['value'])
  label = Literal(result['personLabel']['value'])
  if 'image' in result:
    image = Literal(result['image']['value'])
    print(image)
    g.add((uri, SDO.image, image)) 
  g.add((uri, SDO.name, label))
  g.add((uri, RDF.type, SDO.Person))
  print(label)



[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q925964'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Heidi%20Van%20De%20Vijver%20na%20haar%20overwinning%20in%20het%20BK%20wielrennen%20op%20de%20weg%20in%201994%2C%20Binche%2C%20Maurice%20Terryn%20%28NEGT2865038%2C%20collectie%20KOERS.%20Museum%20van%20de%20Wielersport%29.tif'}, 'personLabel': {'xml:lang': 'nl', 'type': 'literal', 'value': 'Heidi Van De Vijver'}}]
http://commons.wikimedia.org/wiki/Special:FilePath/Heidi%20Van%20De%20Vijver%20na%20haar%20overwinning%20in%20het%20BK%20wielrennen%20op%20de%20weg%20in%201994%2C%20Binche%2C%20Maurice%20Terryn%20%28NEGT2865038%2C%20collectie%20KOERS.%20Museum%20van%20de%20Wielersport%29.tif
Heidi Van De Vijver
[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q234158'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/1991%20World%20Road%20Champion%20-%20Leontien%20van%20Moo

In [31]:
g.serialize(destination="graph.ttl")


<Graph identifier=N2488a1753fd644fa93ce29719108a9b5 (<class 'rdflib.graph.Graph'>)>

In [30]:


def get_info(qid:str):
  query = f"""
  #Wielrenners
  #title: Wielrenners
  SELECT DISTINCT  ?event ?eventLabel ?image  WHERE {{
    BIND(wd:{qid} as ?event)
    OPTIONAL {{
      ?person wdt:p18 ?image
    }}
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "nl,en,fr,mul". }}
  }}
  """


  # Specify the query and format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and get the results
  results = sparql.query().convert()

  # Parse and print the results
  return results["results"]["bindings"]
  for result in results["results"]["bindings"]:
      race = result["raceQid"]["value"]
# Q234158
  # wd:Q2995901
  # wd:Q23701739
  
# Q536470
  # wd:Q3486951
  # wd:Q2488680
  # wd:Q2526723
  # wd:Q27506806
# Q454963
  # wd:Q20107672
  # wd:Q19832815
  # wd:Q2143208
  # wd:Q4703206
# Q925964
  # Q27481596
  #  wd:Q23701739	
pers = {
    'Q234158': ['Q2995901', 'Q23701739'],
    'Q536470': ['Q3486951', 'Q2488680', 'Q2526723', 'Q27506806'],
    'Q454963': ['Q20107672', 'Q19832815', 'Q2143208', 'Q4703206'],
    'Q925964': ['Q27481596', 'Q23701739']
}

for perso in pers.keys():
  print(perso)
  for qid in pers[perso]:
    results = get_info(qid)
    result= results[0]
    qidiri = URIRef(f'http://www.wikidata.org/entity/{perso}')
    uri = URIRef(result['event']['value'])
    label = Literal(result['eventLabel']['value'])
    print(label)
    g.add((uri, SDO.name, label))
    g.add((uri, SDO.participant, qidiri))
    g.add((uri, RDF.type, SDO.Event))




Q234158
1998 UCI Road World Championships – Women's time trial
2004 Tour of Flanders for Women
Q536470
Waalse Pijl 1980
Amstel Gold Race 1976
Ronde van Nederland 1981
1980 Gent–Wevelgem
Q454963
Waalse Pijl 1949
Dwars door Vlaanderen 1945
Ronde van Vlaanderen  1951
Ronde van Vlaanderen  1955
Q925964
2003 Amstel Gold Race (women)
2004 Tour of Flanders for Women
